In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Vertex SDK：AutoML训练文本分类模型用于在线预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_text_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_text_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_text_classification_online.ipynb">
      在Google Cloud Notebooks中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK创建文本分类模型，并使用谷歌云的AutoML模型进行在线预测。您可以访问[AutoML用户指南](https://cloud.google.com/vertex-ai/docs/start/automl-users)。

数据集

本教程使用的数据集是来自Kaggle Datasets的[Happy Moments数据集](https://www.kaggle.com/ritresearch/happydb)。您在本教程中将使用的数据集版本存储在公共云存储桶中。

### 目标

在本教程中，您将使用Vertex SDK从Python脚本创建一个AutoML文本分类模型，并部署以进行在线预测。您也可以使用`gcloud`命令行工具或在Cloud Console中在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到一个服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

费用

本教程使用Google Cloud的计费组件：

- Vertex AI
- Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

###设置本地开发环境

如果您正在使用Colab或Google Cloud笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合本笔记本的要求。您需要以下内容：

- Cloud Storage SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Cloud Storage指南 [设置Python开发环境](https://cloud.google.com/python/setup) 和 [Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了一套简洁的指导：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端中运行 `pip3 install jupyter`命令。

5. 要启动Jupyter，在终端中运行 `jupyter notebook` 命令。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装 Python 的最新版本 Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

也安装最新的GA版本的*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

在安装了额外的包之后，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的 Google Cloud 项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得 $300 的免费信用额度用于计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您将需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行单元格，确保 Cloud SDK 使用正确的项目来执行本笔记本中的所有命令。

**注意**：Jupyter 运行以 `!` 开头的行作为 shell 命令，并且以 `$` 开头的 Python 变量进行插值。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于本笔记本其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不应该使用多地区存储桶来训练Vertex AI。并非所有区域都支持所有Vertex AI服务。

了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行直播教程会话，您可能正在使用共享的测试帐户或项目。为了避免在创建资源时发生用户之间的名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，您的环境已经通过验证。请跳过此步骤。

**如果您正在使用Colab**，请运行下面的单元格，并按照提示进行身份验证以通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入名称，并点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中键入"Vertex"，并选择**Vertex管理员**。在过滤框中键入"Storage Object Admin"，并选择**存储对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务帐户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

当您初始化用于Python的Vertex SDK时，需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源相关的所有数据在会话之间保留的地方。

请在下方设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

为Python项目和对应的存储桶初始化Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在你准备好开始创建自己的AutoML文本分类模型了。

云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的Happy Moments数据集的一个版本，使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来统计示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`TextDataset`类的`create`方法创建`Dataset`资源，参数如下：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件列表，以将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标注模式。

该操作可能需要几分钟时间。

In [ ]:
dataset = aip.TextDataset.create(
    display_name="Happy Moments" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.text.single_label_classification,
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行该管道。

#### 创建训练管道

使用`AutoMLTextTrainingJob`类创建一个AutoML训练管道，具有以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：为模型训练指定的任务类型。
  - `classification`：文本分类模型。
  - `sentiment`：文本情感分析模型。
  - `extraction`：文本实体提取模型。
- `multi_label`：如果是分类任务，指定是否为单标签（False）或多标签（True）。
- `sentiment_max`：如果是情感分析任务，指定最大情感值。

实例化的对象是训练管道的有向无环图（DAG）。

In [ ]:
dag = aip.AutoMLTextTrainingJob(
    display_name="happydb_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
)

print(dag)

#### 运行训练流水线

接下来，通过调用`run`方法并使用以下参数来运行DAG以开始训练作业：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：经过训练的模型的可读名称。
- `training_fraction_split`：用于训练的数据集的百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集的百分比。
- `validation_fraction_split`：用于验证的数据集的百分比。

完成后，`run`方法会返回`Model`资源。

执行训练流水线将需要最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="happydb_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

## 查看模型评估分数
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获得对新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，或者您可以列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=happydb_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

部署模型

接下来，部署您的模型以进行在线预测。要部署模型，您需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

发送在线预测请求

向部署的模型发送在线预测。

获取测试项

您将使用数据集中的一个任意示例作为测试项。不必担心该示例很可能已在训练模型中使用过--我们只是想展示如何进行预测。

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
if len(test_item[0]) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### 进行预测

现在您的`Model`资源已部署到一个`Endpoint`资源，您可以通过向`Endpoint`资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式是：

     { 'content': text_string }

由于predict()方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目的列表发送。

#### 响应

从predict()调用的响应是一个Python字典，包含以下条目：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `displayNames`: 每个类标签的类名。
- `confidences`: 每个类标签的预测置信度，介于0和1之间。
- `deployed_model_id`: 执行预测的部署`Model`资源的Vertex AI标识符。

In [ ]:
instances_list = [{"content": test_item}]

prediction = endpoint.predict(instances_list)
print(prediction)

取消部署模型

当您完成预测工作后，您可以取消部署模型在`Endpoint`资源中的部署。这将取消所有计算资源并终止部署模型的计费。

In [ ]:
endpoint.undeploy_all()

# 清理工作

要清理此项目中使用的所有 Google Cloud 资源，您可以[删除您用于本教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- AutoML 训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud Storage 存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME